In [8]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [9]:
XV = pd.read_csv("../Data/X-values.csv", index_col=[0])
df = XV.drop(["bioactivity_class","Name","canonical_smiles", "molecule_chembl_id"],axis=1)
yc = pd.read_csv("../Data/Y-class.csv", index_col=[0])
yv = pd.read_csv("../Data/Y-values.csv", index_col=[0])

# Importing the data

In [10]:
yv.head(2)

,pIC50
0,5.142668
1,5.026872


In [11]:
o = []
for i in range(len(yv)):
    if (yv.iloc[i][0])>7.0: o.append("Extremely Potent Candidate")
    elif (yv.iloc[i][0])>6.0: o.append("Possible Candidate")
    else: o.append("Unsuitable Candidate")
y = pd.DataFrame(data = o, columns = ['Suitability'])

In [19]:
y

,Suitability
0,Unsuitable Candidate
1,Unsuitable Candidate
2,Unsuitable Candidate
3,Unsuitable Candidate
4,Unsuitable Candidate
...,...
128,Unsuitable Candidate
129,Unsuitable Candidate
130,Unsuitable Candidate
131,Unsuitable Candidate


In [23]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

# 2. FIT
enc.fit(y)

# 3. Transform
onehotlabels = enc.transform(y).toarray()
onehotlabels.shape
y = onehotlabels

In [26]:
y = pd.DataFrame(y)

In [27]:
y

,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
128,0.0,0.0,1.0
129,0.0,0.0,1.0
130,0.0,0.0,1.0
131,0.0,0.0,1.0


In [28]:
y.head(2)

,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,1.0


In [29]:
rem = df[["MW","LogP","NumHDonors","NumHAcceptors"]].copy()
rem.head(1)

,MW,LogP,NumHDonors,NumHAcceptors
0,281.271,1.89262,0.0,5.0


In [30]:
rem = (rem - rem.min()) / (rem.max() - rem.min()) * 2 - 1
print(rem.head(2))

         MW      LogP  NumHDonors  NumHAcceptors
0 -0.598980 -0.451798        -1.0           -0.4
1  0.099774  0.088682        -1.0           -1.0


In [31]:
df1 = df.drop(["MW","LogP","NumHDonors","NumHAcceptors"], axis=1)
df1 = pd.concat([df1,rem], axis=1)
df1.head(3)

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,MW,LogP,NumHDonors,NumHAcceptors
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.598980,-0.451798,-1.0,-0.4
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.099774,0.088682,-1.0,-1.0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.128911,-0.235705,-1.0,-0.6


### Now using only molecular descriptors

In [32]:
X = df1.drop(["MW","LogP","NumHDonors","NumHAcceptors"], axis=1)
# X = pd.concat([X,y], axis = 1)

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,MW,LogP,NumHDonors,NumHAcceptors
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.598980,-0.451798,-1.0,-0.4
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.099774,0.088682,-1.0,-1.0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.128911,-0.235705,-1.0,-0.6
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.536158,0.037352,-1.0,-0.8
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.302073,0.011518,-1.0,-0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.301995,-0.027311,-1.0,-0.4
129,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.520453,-0.015413,-1.0,-0.8
130,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.624888,0.168165,-1.0,-0.8
131,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.614400,-0.058267,-1.0,-0.8


In [37]:
# X = df1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=64)

In [35]:
latent_dim=885

In [42]:
generator = keras.Sequential([
    layers.Dense(256, activation='relu', input_dim=latent_dim),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(df1.shape[1], activation='relu')
])

In [43]:
discriminator = keras.Sequential([
    layers.Dense(512, activation='relu', input_dim=df.shape[1]),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

In [44]:
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [45]:
batch_size = 32
epochs = 100

def generate_noise(n_samples, latent_dim):
    # Generate n_samples of noise vectors, each with latent_dim dimensions
    return np.random.normal(-1, 1, size=(n_samples, latent_dim))

In [46]:
for epoch in range(epochs):
    # Train the discriminator on real and generated data
    real_data = X_train.sample(batch_size)
    noise_data = generate_noise(batch_size, latent_dim)
    generated_data = generator.predict(noise_data)
    x_train = np.concatenate([real_data, generated_data])
    y_train = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    discriminator_loss = discriminator.train_on_batch(x_train, y_train)

    # Train the generator by fooling the discriminator
    noise_data = generate_noise(batch_size, latent_dim)
    y_train = np.ones((batch_size, 1))
    gan_loss = gan.train_on_batch(noise_data, y_train)

    # Print the loss values every 100 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")


1/1 [==============================] - 0s 164ms/step
Epoch: 0, Discriminator Loss: 0.7713680267333984, GAN Loss: 0.9978278279304504
1/1 [==============================] - 0s 35ms/step
Epoch: 10, Discriminator Loss: 6.221828516572714e-05, GAN Loss: 0.2619399428367615
1/1 [==============================] - 0s 27ms/step
Epoch: 20, Discriminator Loss: 0.0003222381928935647, GAN Loss: 0.13357776403427124
1/1 [==============================] - 0s 35ms/step
Epoch: 30, Discriminator Loss: 0.009480729699134827, GAN Loss: 0.2416999340057373
1/1 [==============================] - 0s 35ms/step
Epoch: 40, Discriminator Loss: 0.000398231582948938, GAN Loss: 0.605305552482605
1/1 [==============================] - 0s 35ms/step
Epoch: 50, Discriminator Loss: 0.0023073938209563494, GAN Loss: 1.0163980722427368
1/1 [==============================] - 0s 29ms/step
Epoch: 60, Discriminator Loss: 0.013508105650544167, GAN Loss: 1.5177934169769287
1/1 [==============================] - 0s 37ms/step
Epoch: 70

In [ ]:
generator2 = keras.Sequential([
    layers.Dense(256, activation='relu', input_dim=latent_dim),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(df1.shape[1], activation='relu')
])

discriminator2 = keras.Sequential([
    layers.Dense(512, activation='relu', input_dim=df.shape[1]),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

discriminator2.compile(optimizer='adam', loss='binary_crossentropy')
discriminator2.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan2 = keras.models.Model(gan_input, gan_output)
gan2.compile(optimizer='adam', loss='binary_crossentropy')

batch_size = 32
epochs = 100

def generate_noise(n_samples, latent_dim):
    # Generate n_samples of noise vectors, each with latent_dim dimensions
    return np.random.normal(-1, 1, size=(n_samples, latent_dim))

In [47]:
n_samples = 100
noise_data = generate_noise(n_samples, latent_dim)
generated_data = generator.predict(noise_data)

4/4 [==============================] - 0s 4ms/step


In [48]:
df1.describe()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,MW,LogP,NumHDonors,NumHAcceptors
count,133.000000,133.000000,133.000000,133.000000,133.0,133.0,133.0,133.0,133.0,133.0,...,133.0,133.0,133.0,133.0,133.0,133.0,133.000000,133.000000,133.000000,133.000000
mean,0.992481,0.932331,0.413534,0.007519,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.104135,0.115422,-0.393484,-0.350376
std,0.086711,0.252127,0.494329,0.086711,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.453748,0.389912,0.588662,0.414441
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.520453,-0.078163,-1.000000,-0.800000
50%,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.139914,0.076497,-0.333333,-0.400000
75%,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.213432,0.328988,0.333333,-0.200000
max,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000


In [49]:
print(df1.max())

PubchemFP0       1.0
PubchemFP1       1.0
PubchemFP2       1.0
PubchemFP3       1.0
PubchemFP4       0.0
                ... 
PubchemFP880     0.0
MW               1.0
LogP             1.0
NumHDonors       1.0
NumHAcceptors    1.0
Length: 885, dtype: float64


In [99]:
df1.min()

PubchemFP0       0.0
PubchemFP1       0.0
PubchemFP2       0.0
PubchemFP3       0.0
PubchemFP4       0.0
                ... 
PubchemFP880     0.0
MW              -1.0
LogP            -1.0
NumHDonors      -1.0
NumHAcceptors   -1.0
Length: 885, dtype: float64

In [50]:
g = generated_data
print(g[0][:])

[-5.57940066e-01  4.34216231e-01  3.67630213e-01  8.45048606e-01
  8.36533129e-01 -7.03467906e-01  2.37639740e-01  9.07000661e-01
 -9.91991699e-01  3.20937455e-01 -4.93582003e-02 -4.17240977e-01
  2.31611311e-01  9.08447921e-01 -3.13532352e-01  1.22922786e-01
 -7.64945149e-01  8.01993966e-01  9.04072702e-01  9.12710607e-01
  1.87574267e-01  2.36118555e-01  9.20590200e-03  8.06705713e-01
 -6.94019556e-01  2.10346237e-01 -7.27296174e-01 -6.91738307e-01
  7.45602369e-01  8.37937295e-01  1.35474578e-01  8.57891977e-01
 -5.53228438e-01  5.66802263e-01 -1.53542250e-01  8.63536656e-01
  2.73019463e-01  9.15188313e-01  9.48139608e-01 -7.27322578e-01
 -6.77557409e-01 -7.34660029e-02 -7.12282881e-02  7.93157756e-01
  7.11727589e-02  2.42008902e-02  5.26500046e-01 -6.40468299e-02
 -4.38532859e-01  6.34671748e-01  7.63174966e-02 -1.33538052e-01
 -2.94775456e-01  4.45769668e-01  1.24374464e-01  6.47543669e-01
 -6.09224677e-01  6.73312098e-02 -2.84997791e-01  9.13155258e-01
  1.62349582e-01  1.16463

In [52]:
X_train.loc[0][:]

PubchemFP0       1.000000
PubchemFP1       1.000000
PubchemFP2       0.000000
PubchemFP3       0.000000
PubchemFP4       0.000000
                   ...   
PubchemFP880     0.000000
MW              -0.598980
LogP            -0.451798
NumHDonors      -1.000000
NumHAcceptors   -0.400000
Name: 0, Length: 885, dtype: float64

In [31]:
type(g)

numpy.ndarray

In [94]:
g.size

88500

In [89]:
generated_data = (generated_data + 1) / 2 * (df1.max() - df1.min()) + df1.min()

ValueError: Length of values (100) does not match length of index (885)